In [323]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
from steam.api import interface
import time
steam.api.key.set('9D3E7E9815B466DACB9B11D45C4E7E50')

In [324]:
#declare interface urls
#dota2_beta should be used for testing in order to not aggressively make API requests
dota2_id = '570'
dota2_beta_id = '205790'

# Get 500 last matches to get approximate hero ratios

In [279]:
#get matches data from dota2 API and put IDs into a dataframe
num_matches_to_request = 500

match_list = []
matches = interface('IDOTA2Match_' + dota2_id).GetMatchHistory(game_mode = 1, skill_level = 3, min_players = 10, matches_requested = 100)

#@TODO: check if lobby_type is public/ranked matching (0 or 7) before appending
[match_list.append(match['match_id']) for match in matches['result']['matches']]

current_num = 100

while current_num <= num_matches_to_request:
    #sleep so we don't overload the servers
    time.sleep(1)

    current_num += 100
    matches = interface('IDOTA2Match_' + dota2_id).GetMatchHistory(start_at_match_id = match_list[-1]-1, game_mode = 1,
                                                              skill_level = 3, min_players = 10, matches_requested = 100)

    #@TODO: check if lobby_type is public/ranked matching (0 or 7) before appending
    [match_list.append(match['match_id']) for match in matches['result']['matches']]

    
dota2_validation_df = pd.DataFrame()
dota2_validation_df['match_id'] = match_list

In [281]:
#get match details from match_ids obtained
#add radiant_win column
#add heroes split between dire and radiant

radiant_win_column = []
radiant_hero_column = []
dire_hero_column = []
for match_id in dota2_validation_df['match_id']:
    time.sleep(1)
    match_details = interface('IDOTA2Match_' + dota2_id).GetMatchDetails(match_id = str(match_id))
    
    radiant_win_column.append(match_details['result']['radiant_win'])
    
    radiant_hero_row = []
    dire_hero_row = []
    for player in match_details['result']['players']:
        #First bit of 8-bit integer player_slot is 1 if the player was on dire
        #so if player was on dire then player_slot > 128
        if player['player_slot']/128<1:
            radiant_hero_row.append(player['hero_id'])
        else:
            dire_hero_row.append(player['hero_id'])
    
    radiant_hero_column.append(radiant_hero_row)
    dire_hero_column.append(dire_hero_row)

In [291]:
#Add radiant_win column to Dataframe
dota2_validation_df['radiant_win'] = 1*np.array(radiant_win_column)

#Add dire and radiant heroes to DataFrame under their own columns
for i in range(len(np.array(radiant_hero_column[1]))):
    dota2_validation_df['radiant_heroes'+str(i)] = np.array(radiant_hero_column)[:,1]
    dota2_df['dire_heroes'+str(i)] = np.array(dire_hero_column)[:,1]

0
1
2
3
4


In [293]:
dota2_validation_df.to_csv('dota2_validation_data.csv')

# Get 200 matches of each hero id (and trim duplicates)

In [299]:
hero_list = interface('IEconDOTA2_'+dota2_id).GetHeroes(itemizedonly = True)

In [306]:
#get matches data from dota2 API and put IDs into a dataframe
num_matches_to_request = 200

#Loop through all hero_id values
#There are 116 heroes, but ids 105, 116-118 are not occupied
for i in range(hero_list['result']['count']):
    
    #better option is to build a vocabulary
    if i == 105:
        j = 121
    elif i == 115:
        j = 120
    elif i == 116:
        j = 119
    else:
        j = i

    #Get first match for each hero out of loop so that we can pull more than 100 games per hero
    #by utilizing start_at_match_id option
    matches = interface('IDOTA2Match_' + dota2_id).GetMatchHistory(hero_id = j, game_mode = 1, skill_level = 3,
                                                              min_players = 10, matches_requested = 1)
    
    #@TODO: check if lobby_type is public/ranked matching (0 or 7) before appending
    [match_list.append(match['match_id']) for match in matches['result']['matches']]
    current_num = 1
    
    while current_num <= num_matches_to_request:
        #sleep so we don't overload the servers
        time.sleep(1)

        current_num += 100
        matches = interface('IDOTA2Match_' + dota2_id).GetMatchHistory(hero_id = j,
                                                    start_at_match_id = match_list[-1]-1,game_mode = 1,
                                                    skill_level = 3, min_players = 10, matches_requested = 100)
        
        #@TODO: check if lobby_type is public/ranked matching (0 or 7) before appending
        [match_list.append(match['match_id']) for match in matches['result']['matches']]

In [320]:
#Make dataframe of match_id values using dict.fromkeys(list) to remove duplicates
dota2_training_df = pd.DataFrame()
dota2_training_df['match_id'] = list(dict.fromkeys(match_list))

In [333]:
#get match details from match_ids obtained
#add radiant_win column
#add heroes split between dire and radiant

#radiant_win_column = []
#radiant_hero_column = []
#dire_hero_column = []

for i,match_id in enumerate(dota2_training_df['match_id'][266:]):
    time.sleep(1)
    match_details = interface('IDOTA2Match_' + dota2_id).GetMatchDetails(match_id = str(match_id))
    
    try:
        radiant_win_column.append(match_details['result']['radiant_win'])
    except:
        print('server error at match_id:' + str(match_id))
        print('element:' + str(i))
        break
    
    radiant_hero_row = []
    dire_hero_row = []
    for player in match_details['result']['players']:
        #First bit of 8-bit integer player_slot is 1 if the player was on dire
        #so if player was on dire then player_slot > 128
        if player['player_slot']/128<1:
            radiant_hero_row.append(player['hero_id'])
        else:
            dire_hero_row.append(player['hero_id'])
    
    radiant_hero_column.append(radiant_hero_row)
    dire_hero_column.append(dire_hero_row)

In [367]:
#Add radiant_win column to Dataframe
dota2_training_df['radiant_win'] = 1*np.array(radiant_win_column)

#Add dire and radiant heroes to DataFrame under their own columns
for i in range(len(np.array(radiant_hero_column)[1])):
    print(i)
    dota2_training_df['radiant_heroes'+str(i)] = np.array(radiant_hero_column)[:,i]
    dota2_training_df['dire_heroes'+str(i)] = np.array(dire_hero_column)[:,i]

0
1
2
3
4


In [368]:
dota2_training_df.to_csv('dota2_training_data.csv')

7051